<a href="https://colab.research.google.com/github/rakeshgovindsuresh/Python-Practice/blob/main/flasktry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

# Load data
df = pd.read_csv('/content/Combined no2 dataset (1).csv')

# Parse datetime and extract features
df['datetimeUtc'] = pd.to_datetime(df['datetimeUtc'])
df['hour'] = df['datetimeUtc'].dt.hour

# Classify period of the day
def classify_period(hour):
    if 6 <= hour < 14:
        return 'Morning'
    elif 14 <= hour < 22:
        return 'Evening'
    else:
        return 'Night'

df['TimeOfDay'] = df['hour'].apply(classify_period)


In [3]:
def classify_no2(value):
    if value < 30:  # hypothetical threshold
        return 'Good'
    elif 30 <= value < 65:  # hypothetical threshold
        return 'Neutral'
    else:
        return 'Dangerous'

df['NO2_Level'] = df['value'].apply(classify_no2)


In [4]:
# Encode categorical data
label_encoder = LabelEncoder()
df['TimeOfDay'] = label_encoder.fit_transform(df['TimeOfDay'])

# Features and Target
X = df[['TimeOfDay', 'hour']]  # add more features as needed
y = df['NO2_Level']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [5]:
# Create a Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
clf.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [6]:
# Predictions
y_pred = clf.predict(X_test)

# Evaluation
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

   Dangerous       0.00      0.00      0.00        72
        Good       0.72      1.00      0.84       770
     Neutral       0.00      0.00      0.00       232

    accuracy                           0.72      1074
   macro avg       0.24      0.33      0.28      1074
weighted avg       0.51      0.72      0.60      1074



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
pip install flask


In [12]:
from flask import Flask, render_template, request, redirect, url_for
import folium
import pandas as pd

app = Flask(__name__)

# Assuming your data is already loaded and processed as per the previous steps:
df = pd.read_csv('/content/Combined no2 dataset (1).csv')
df['datetimeUtc'] = pd.to_datetime(df['datetimeUtc'])
df['hour'] = df['datetimeUtc'].dt.hour
df['day'] = df['datetimeUtc'].dt.day_name()  # Adding day names

# Function to create map
def create_day_map(filtered_data):
    start_coords = (filtered_data['latitude'].mean(), filtered_data['longitude'].mean())
    folium_map = folium.Map(location=start_coords, zoom_start=6)
    colors = {'Dangerous': 'red', 'Neutral': 'yellow', 'Good': 'green'}
    for _, row in filtered_data.iterrows():
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=5,
            color=colors.get(row['NO2_Level'], 'blue'),  # Default to blue if undefined
            fill=True,
            fill_color=colors.get(row['NO2_Level'], 'blue'),
            popup=f"{row['location_name']}: {row['NO2_Level']}"
        ).add_to(folium_map)
    return folium_map

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/map/<day>')
def day_map(day):
    # Filter data for the selected day and 'Morning'
    day_data = df[(df['day'] == day) & (df['TimeOfDay'] == 'Morning')]
    folium_map = create_day_map(day_data)
    map_file = f'static/{day}_map.html'
    folium_map.save(map_file)
    return render_template('map.html', map_file=map_file)

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>NO2 Map</title>
</head>
<body>
    <h1>NO2 Levels Map</h1>
    <iframe src="{{ map_file }}" width="100%" height="600"></iframe>
    <br>
    <button onclick="location.href='/'">Back to Days</button>
</body>
</html>

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Select a Day</title>
</head>
<body>
    <h1>Select a Day to Display NO2 Levels</h1>
    {% for day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'] %}
    <button onclick="location.href='/map/{{ day }}'">{{ day }}</button>
    {% endfor %}
</body>
</html>
